In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
from pprint import pprint
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr
import shapely as shp
import os
import re
import string
import math

In [ ]:
def geometryToEE(img_geometry):
    
    # format geometry
#     img_geometry = gpd.GeoDataFrame(geometry = img_geometry)
    print(img_geometry)
    img_geometry = img_geometry.reset_index(drop = True)
    img_geometry = [[[x, y] for x, y in list(img_geometry.geometry.exterior[0].coords)]]
    
    # convert geometry to ee.Geometry
    img_geometry_ee = ee.Geometry({
        'type': 'Polygon',
        'coordinates': img_geometry
    })
    
    return img_geometry_ee

In [ ]:
# Read in RTS polygons
polys = (ee.FeatureCollection('users/gfiske/FrostCraters/rts_polygons_for_Yili_May_2022')
         .filter(ee.Filter.bounds(ee.Geometry.BBox(65, 68, 81, 73))));

In [ ]:
# Prep Map
Map = geemap.Map()
Map.setCenter(73, 70.5, 7);

In [ ]:
# Get Sentinel-2 Mosaic to Align Planet Data
MAX_CLOUD_PROBABILITY = 50
# geometry = ee.Geometry({
#     'type': 'Polygon',
#     'coordinates': [[
#         [66, 66],
#         [66, 73.5],
#         [85, 73.5],
#         [85, 66],
#         [66, 66]
#         ]]
# })

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
#     .filterBounds(geometry)
    .filterDate('2018-06-15', '2019-08-31')
    .filter(ee.Filter.dayOfYear(182, 243))
);

s2_clouds = (
    ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
#     .filterBounds(geometry)
    .filterDate('2018-06-15', '2019-08-31')
    .filter(ee.Filter.dayOfYear(182, 243))
);

def s2_maskcloud(image):
    
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    
    return image.updateMask(isNotCloud)

# Join S2 SR with cloud probability dataset to add cloud mask.
s2_with_clouds = ee.Join.saveFirst('cloud_mask').apply(    
  primary=s2,
  secondary=s2_clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

s2_masked = ee.ImageCollection(s2_with_clouds).map(s2_maskcloud)

# s2_masked_north = s2_masked.filter(ee.Filter.dayOfYear(196, 243))

s2_mosaic = (
    s2_masked
    .select(['B2', 'B3', 'B4', 'B8'])
    .median());

# s2_mosaic_north = (
#     s2_masked_north
#     .select(['B2', 'B3', 'B4', 'B8'])
#     .median());

pprint(s2_mosaic.getInfo())

In [ ]:
# Export only in the polygons we need
# Import shapefile with AOI (multipolygon)
aoi = gpd.read_file("/home/hrodenhizer/Documents/permafrost_pathways/rts_mapping/planet_processing_test/data/yg_val_regions/bboxes/yg_validation_bboxes.shp")
# convert from multipolygon to multiple polygons
aoi = aoi.explode(column = 'geometry', ignore_index = True)
# remove inner holes
aoi.geometry = aoi.geometry.exterior
# convert back to polygon
aoi.geometry = [shp.geometry.Polygon([shp.geometry.Point(x, y) for x, y in list(feature.coords)]) for feature in aoi.geometry]
aoi.geometry = aoi.geometry.to_crs(crs = 32642).buffer(1000, join_style = 2).to_crs(crs = 4326)
pprint(aoi)

# convert to ee.Geometry objects
aoi_ee_0 = geometryToEE(aoi[0:1])
aoi_ee_1 = geometryToEE(aoi[1:2])
aoi_ee_2 = geometryToEE(aoi[2:3])
aoi_ee_3 = geometryToEE(aoi[3:4])

In [ ]:
Map.addLayer(s2_mosaic.clip(aoi_ee_0), 
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1000}, 
             'S2 All Years, Region 1')
Map.addLayer(s2_mosaic.clip(aoi_ee_1), 
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1200}, 
             'S2 All Years, Region 2')
Map.addLayer(s2_mosaic.clip(aoi_ee_2), 
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1200}, 
             'S2 All Years, Region 3')
Map.addLayer(s2_mosaic.clip(aoi_ee_3), 
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1200}, 
             'S2 All Years, Region 4')
Map.addLayer(polys,
             {'color': 'red'},
             'RTS Polygons')

In [ ]:
Map

In [ ]:
def utm_from_wgs84(lon, lat):
    #Special Cases for Norway and Svalbard
    if (lat > 55 and lat < 64 and lon > 2 and lon < 6):
        return 32
    elif (lat > 71 and lon >= 6 and lon < 9):
        return 31
    elif (lat > 71 and ((lon >= 9 and lon < 12) or (lon >= 18 and lon < 21))):
        return 33
    elif (lat > 71 and ((lon >= 21 and lon < 24) or (lon >= 30 and lon < 33))):
        return 35
    # Rest of the world
    elif (lon >= -180 and lon <= 180):
        return 32600 + (math.floor((lon + 180) / 6) % 60) + 1 # 32600 for northern hemisphere
    else:
        raise ValueError('Cannot figure out UTM zone from given Lat: {0}, Lon: {1}.'.format(lat, lon))

zones = pd.DataFrame(columns = ['region', 'utm_zone'])
for idx, region in enumerate(aoi.geometry):
    region_zones = []
    for x, y in zip(region.exterior.coords.xy[0], region.exterior.coords.xy[1]):
        region_zones.append(utm_from_wgs84(x, y))
        
    region_zones = set(region_zones)
    temp_df = pd.DataFrame({'region': [idx]*len(region_zones),
                            'utm_zone': list(region_zones)})
    zones = pd.concat([zones, temp_df])
zones = zones.set_index('region')  
zones

In [ ]:
aoi = pd.merge(aoi, zones, left_index = True, right_index = True)
aoi

In [ ]:
for row in aoi.iterrows():
    polygon_id = row[0]
    description = 'Sentinel-2_YG_Validation_mosaic_bgrnir_Region_' + str(row[0]) + '_UTM' + str(row[1]['utm_zone']-32600) + 'N'
    name = 'planet_processing/data/yg_val_regions/sentinel_data/Sentinel-2_YG_Validation_mosaic_bgrnir_Region_' + str(row[0]) + '_UTM' + str(row[1]['utm_zone']-32600) + 'N'
    img_geometry = [[[x, y] for x, y in list(row[1]['geometry'].exterior.coords)]]
    
    crs = 'EPSG:' + str(row[1]['utm_zone'])
   
    # Export 2018-2019 growing season Sentinel-2 Mosaic to Drive
    task = ee.batch.Export.image.toCloudStorage(
        image = s2_mosaic,
        description = description,
        bucket = 'abrupt_thaw',
        fileNamePrefix = name,
        crs = crs,
        scale = 10,
        maxPixels = 1e13,
        region = img_geometry,
        fileFormat = 'GeoTIFF',
        formatOptions = {
            'cloudOptimized': True
        }
    )
    task.start()